# Combine: Intercity Passenger Rail Service Station Performance Metrics

This notebook reads a series of Excel files containing [Amtrak](https://www.amtrak.com/home.html)
station performance metrics, combines the data into a single [pandas](https://pandas.pydata.org/)
`DataFrame`, and writes the combined dataset to a CSV file for follow up cleaning, manipulation,
and analysis.

### Variable names

A number of variable names in this project leverage the following abbreviations. The naming
strategy is to strike a balance between brevity and readability:

* `amtk`: Amtrak (reporting mark)
* `chrt`: chart
* `cols`: columns
* `const`: constant
* `cwd`: current working directory
* `eb`: eastbound direction of travel
* `lm`: linear model
* `mi`: miles
* `mm`: minutes (ISO 8601)
* `nb`: northbound direction of travel
* `psgr`: passenger
* `qtr`: quarter
* `rte`: route
* `sb`: southbound direction of travel
* `stats`: summary statistics
* `stn`: station
* `stns`: stations
* `svc`: service
* `trn`: train
* `wb`: westbound direction of travel

In [ ]:
import numpy as np
import pandas as pd
import pathlib as pl
import tomllib as tl

import fra_amtrak.amtk_frame as frm

## 1.0 Read files

### 1.1 Resolve paths

Instantiate instances of `pathlib.Path` to represent absolute paths to the `data/raw` and `data/interim` directories.

In [ ]:
parent_path = pl.Path.cwd()  # current working directory
parent_path

data_raw_path = parent_path.joinpath("data", "raw")
data_interim_path = parent_path.joinpath("data", "interim")

### 1.2 Load constants

Load a companion [TOML](https://toml.io/en/) file named `notebook.toml` containing constants.

In [ ]:
filepath = parent_path.joinpath("notebook.toml")
with open(filepath, "rb") as file_obj:
    const = tl.load(file_obj)

# Access constants
COLS = const["columns"]

### 1.3 Read data files

The FRA's station performance filenames are inconsistent (review the files in the `data/raw`
directory). The [`Path.glob()`](https://docs.python.org/3/library/pathlib.html#pathlib.Path.glob)
method is used to retrieve the otherwise irregular filepaths via pattern matching.

In [ ]:
# Check filepaths (this can also be done with the os or glob modules)
filepaths = data_raw_path.glob("*Station%20Performance*.xlsx")
filepaths = [filepath for filepath in filepaths if filepath.is_file()]

print(f"filepaths (n={len(filepaths)})")

Below are the preferred data types for each column in the dataset:

In [ ]:
# Specify dtypes
dtypes = {
    "Fiscal Year": np.int16,
    "Fiscal Quarter": np.int8,
    "Service Line": "string",
    "Service": "string",
    "Sub Service": "string",
    "Train Number": np.int16,
    "Arrival Station Code": "string",
    "Arrival Station Name": "string",
    "Total Detraining Customers": np.int32,
    "Late Detraining Customers": np.int32,
    "Avg Min Late (LT CS)": np.int32,
    "Avg Min Late (LT C)": np.int32,
}


### 1.4 Combine `DataFrames` [1 pt]

The next task is to use pandas to read the station peformance `*.xlsx` files located in the
`data/raw` directory into a set of unnamed `DataFrame` objects, specifying specific column `dtypes`
for each using a dictionary, and ignoring the indexes.  The `DataFrame` objects are then combined
into a single `DataFrame` object named `stations`.

Note: this operation can be performed in a single line of code, although length considerations will
dictate the use of multiple lines.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

In [ ]:
stations.info()

In [ ]:
stations.head()

## 2.0 Drop "Unnamed" column

Each Excel spreadsheet features an unnamed first column consisting of no row values. Drop the column.

In [ ]:
# Print unique values in Unnamed column
mask = stations.columns[stations.columns.str.contains("^Unnamed")]
print(f"Unnamed columns (n={len(mask)}):")

# Print unique values in unnamed column
unnamed_unique_values = stations[mask].apply(lambda x: x.unique())
print(f"unnamed_unique_values = {unnamed_unique_values}")

# Drop Unnamed column
stations = stations.loc[:, ~stations.columns.str.contains("^Unnamed")]
stations.head()

## 3.0 Transform mixed type columns

Check columns for the presence of mixed types. The goal is to avoid triggering a `DtypeWarning` when
reading the combined dataset into a `DataFrame` in subsequent notebooks.

```commandline
DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  stations = pd.read_csv(filepath, dtype=dytpes)
```

Suspect columns include:

* Avg Min Late (Lt CS)
* Avg Min Late (Lt C)

If strings are present in these columns, convert them to `NaN` and then recast the columns to
`np.float32`.

### 3.1 Traverse the columns

In [ ]:
# Traverse data frame to detect data types
for column in stations.columns:
    print(f"{column}: ", pd.api.types.infer_dtype(stations[column]))

### 3.2 Identify the non-numeric values


Call the function `frm.find_non_numeric_values()` and return the non-numeric values polluting the
"Avg Min Late" columns.

In [ ]:
non_numeric_values = {
    column: frm.find_non_numeric_values(stations, column) for column in stations.columns[-2:]
}
non_numeric_values

Return a count of the string values (`"--"`) in the "Avg Min Late" columns.

In [ ]:
lt_cs_dashes_count = stations.loc[:, COLS["avg_mm_late_cs"]].str.contains("--").sum()
print(f"LT CS dashes count = {lt_cs_dashes_count}")

lt_c_dashes_count = stations.loc[:, COLS["avg_mm_late_c"]].str.contains("--").sum()
print(f"LT C dashes count = {lt_c_dashes_count}")

### 3.1 Convert dashes to `NaN` [1 pt]

Convert dashes (`--`) in both columns to `NaN` and then convert the columns to `np.float32`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

Confirm the conversion by checking the `dtype` of each column in `stations`. Note the reduction in memory usage as a result of careful type selection.

In [ ]:
stations.info()

## 4.0 Sort data [1 pt]

Sort `stations` by its first seven (`7`) columns, ordered as follows:

1. "Fiscal Year" (descending)
2. "Fiscal Quarter" (descending)
3. "Service Line" (ascending)
4. "Service" (ascending)
5. "Sub Service" (ascending)
6. "Train Number" (ascending)
7. "Arrival Station Code" (ascending)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

## 5.0 Check years, quarters covered

### 5.1 Fiscal years and quarters [1 pt]

Group `stations` on the fiscal year and fiscal quarter, and return a count of the number of rows
associated with each year-quarter combination. Sort the rows in descending order, reset the index,
and assign the new column the name "Rows." Assign the new `DataFrame` the name `periods`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

### 5.2 Service lines [1 pt]

Group `stations` on the fiscal year, fiscal quarter, and service line and return a count of the number
of rows associated with each year-quarter-service line combination. Sort the rows in descending
order, reset the index, and assign the new column the name "Rows." Assign the new `DataFrame` the
name `periods`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#hidden tests are within this cell

## 6.0 Persist data

### 6.1 Recheck data

In [ ]:
stations.info()

### 6.2 Write to file [1 pt]

Write data to a CSV file.

In [ ]:
filepath = data_interim_path.joinpath("station_performance_metrics-v1p0.csv")
stations.to_csv(filepath, index=False)

In [ ]:
#hidden tests are within this cell

## 6.0 Watermark

In [ ]:
%load_ext watermark
%watermark -h -i -iv -m -v